# HE-BCI

Here we collect data from the Higher-Education Business Community Interaction Survey available from the HESA website ([link](https://www.hesa.ac.uk/data-and-analysis/business-community))

The structure is very similar to other HESA we collected in `01_jmg` so eventually we might want to merge both notebooks. I will definitely be reusing a lot of the code here.

In terms of indicators, we would like to create the following:

* Graduate start-ups rate (HE-BCI)
* Research resource (income) per spin-out (HE-BCI)
* Average external investment per formal spin-out (HE-BCI)
* Licensing and other IP income as proportion of research income (HE-BCI)
* Contract research income with businesses (HE-BCI)
* Consultancy income with businesses (HE-BCI)
* Contract research income with the public and third sector (HE-BCI)
* Consultancy income with the public and third sector  (HE-BCI)

## Preamble

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
import csv
import zipfile
import io

In [ ]:
import seaborn as sn
from nuts_finder import NutsFinder

In [ ]:
today_str = str(datetime.datetime.today()).split(' ')[0]

### Functions

#### Simple utilities

In [ ]:
def tidy_cols(my_csv):
    '''
    Tidies column names ie lower and replace spaces with underscores
    
    '''
    
    return([re.sub(' ','_',col.lower()) for col in my_csv.columns])

In [ ]:
def filter_data(data,var_val_pairs):
    '''
    We use this to filter the data more easily than using pandas subsetting
    
    Args:
        data (df) is a dataframe
        var_val pairs (dict) is a dictionary where the keys are variables and the value are values

    
    '''
    d = data.copy()
    
    for k,v in var_val_pairs.items():
        d = d.loc[d[k]==v]
        
    return(d.reset_index(drop=True))
    

In [ ]:
def check_categories(data,columns):
    '''
    This counts frequencies of categorical variables. We use it to decide what variables to choose, and to avoid double counting
    
    Args:
        Data (df) is the data
        Columns (list) are the categorical variables we want to check
    
    '''
    print('FREQUENCIES')
    print('===========')
    
    print('\n')
    #We check frequencies
    
    for var in columns:
    
        print(var)
        print('=====')
        print(data[var].value_counts())

        print('\n')
        
    print('CROSSTABS')
    print('===========')
    
    #We check combinations
    
    combs = list(combinations(columns,2))
    
    for comb in combs:
        print(comb[0]+' x '+comb[1])
        print('=====')
        print(pd.crosstab(data[comb[0]],data[comb[1]]))
        
        print('\n')
        
        
    


#### Data collection

In [ ]:
def hesa_parser(url,out_name,skip=16,encoding='utf-8'):
    '''
    Function to obtain and parse data from the HESA website 
    
    Args:
        url (str) is the location of the csv file
        out_name (str) is the saved name of the file
        skip is the number of rows to skip (we could automate this by looking for rows at the top with lots of nans)
    
    '''
    
    #Request and parse
    rs = requests.get(url)
    
    #Parse the file
    parsed = rs.content.decode(encoding)
    
    #Save it
    
    with open(f'../../data/raw/hesa/{out_name}.txt','w') as outfile:
        outfile.write(parsed)
        
    #Read it.
    my_csv = pd.read_csv(f'../../data/raw/hesa/{out_name}.txt',skiprows=skip)
    
    #Clean column names
    my_csv.columns = tidy_cols(my_csv)
    
    
    return(my_csv)

    
    

#### Data processing

In [ ]:
def gimme_nuts(lat,lon,level=2):
    '''
    Function to extract nuts information from a pair of coordinates
    
    Args:
        lat (float) is the latitude
        lon (float) is the longitude
        level (int) is the NUTS level we want
        
    
    '''
    
    info = nf.find(lat=lat,lon=lon)
    try:
        nuts_id = [x['NUTS_ID'] for x in info if x['LEVL_CODE']==level][0]
        nuts_name = [x['NUTS_NAME'] for x in info if x['LEVL_CODE']==level][0]
    #print(info)
    
    #nuts_id = info[level]['NUTS_ID']
    #nuts_name = info[level]['NUTS_NAME']
    
    except:
        print(f'failed with {np.round(lat,2)},{np.round(lon,2)}')
        nuts_id = np.nan
        nuts_name=np.nan
    
    return([nuts_id,nuts_name])

In [ ]:
def compare_data(df_1,df_2,id_1,id_2,name_1,name_2):
    '''
    We use this function to check if the ids in two datasets we are merging are consistent.
    
    Args:
        dfs are the dfs we want to compare
        ids are the ids we want to check
        names are the names we want to use to explore the data
    
    '''
    
    print('In 1 but not in 2')
    print('==================')
    d1_miss = set(df_1[id_1].dropna())-set(df_2[id_2])
    print(set(df_1.loc[[x in d1_miss for x in df_1[id_1]]][name_1]))
    
    print('\n')

    
    print('In 2 but not in 1')
    print('==================')
    d2_miss = set(df_2[id_2].dropna())-set(df_1[id_1])
    print(set(df_2.loc[[x in d2_miss for x in df_2[id_2]]][name_2]))

    

#### Create NUTS aggregations

In [ ]:
def make_nuts_estimate(data,nuts_lookup,counter,name):
    '''
    This function takes hesa data and creates a nuts estimate
    
    Args:
        data (df) where we have already selected variables of interest eg mode of employment
        nuts (dict) is the ukprn - nuts name and code lookup
        counter (str) is the variable with counts that we are interested in
    
    '''
    
    d = data.copy()
    
    #Add the nuts names and codes
    d['nuts_name'],d['nuts_code'] = [[nuts_lookup[ukprn][var] if ukprn in nuts_lookup.keys() else np.nan for ukprn in data['ukprn']] for
                                     var in ['nuts_name','nuts_code']]
    
    #We are focusing on numbers
    d[counter] = d[counter].astype(float)
    
    out = d.groupby(['nuts_name','nuts_code'])[counter].sum()
    
    out.name = name
    
    return(out)

In [ ]:
def multiple_nuts_estimates(data,nuts_lookup,variables,select_var,value):
    '''
    Creates NUTS estimates for multiple variables.
    
    Args:
        data (df) is the filtered dataframe
        select_var (str) is the variable we want to use to select values
        nuts_lookup (dict) is the lookup between universities and nuts
        variables (list) is the list of variables for which we want to generate the analysis
        value (str) is the field that contains the numerical value we want to aggregate in the dataframe
    
    '''
    
    concat = pd.concat([make_nuts_estimate(data.loc[data[select_var]==m],nuts_lookup,value,m) for m in 
              variables],axis=1)
    
    return(concat)
        
    

#### Directories etc

In [ ]:
# Create a hesa directory in raw and processed

In [ ]:
if 'hesa' not in os.listdir('../../data/raw'):
    os.mkdir('../../data/raw/hesa')
    
if 'hesa' not in os.listdir('../../data/processed'):
    os.mkdir('../../data/processed/hesa')

## 1. Collect data

### University metadata

The [learning providers website](http://learning-provider.data.ac.uk/) contains information about universities.



In [ ]:
uni_meta = pd.read_csv('http://learning-provider.data.ac.uk/data/learning-providers-plus.csv')

In [ ]:
uni_meta.columns = tidy_cols(uni_meta)

In [ ]:
uni_meta.head()

Label universities

In [ ]:
#This initialises an object to label lons and lats with their NUTS code
nf = NutsFinder(scale=1)

In [ ]:
#Create a dict from ukprn to name, nuts2
uni_nuts = {row['ukprn']:{'name':row['view_name'],
                          'nuts_code':gimme_nuts(lat=row['latitude'],lon=row['longitude'])[0],
                          'nuts_name':gimme_nuts(lat=row['latitude'],lon=row['longitude'])[1]} for rid,row in uni_meta.iterrows()}

### Spin-out activity

In [ ]:
url_1 = 'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-4e.csv'

In [ ]:
spin = hesa_parser(url_1,'spin',skip=11)

In [ ]:
spin.head()

### Licensing income

In [ ]:
url_2 = 'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-4d.csv'

In [ ]:
ip = hesa_parser(url_2,'ip',skip=11)

In [ ]:
ip.head()

### Services income

In [ ]:
url_3 = 'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-2a.csv'

In [ ]:
services = hesa_parser(url_3,'services',skip=11)

In [ ]:
services.head()

### Collaborative research involving public funding

In [ ]:
url_4 = 'https://www.hesa.ac.uk/data-and-analysis/providers/business-community/table-1.csv'

In [ ]:
collab = hesa_parser(url_4,'collab',skip=11)

In [ ]:
collab.head()

### Coverage

We want to check where there are mismatches in the institutions in both datasets.

In [ ]:
for data,name in zip([spin,ip,services,collab],['spin','ip','services','collab']):
    
    print(name)
    print('======')
    print(compare_data(data,uni_meta,'ukprn','ukprn','he_provider','view_name'))
    
    print('\n')

This is similar to what we got in `1` - a small number of specialist institutions are missing from the HESA data. 

## 2. Create indicators

### spinout related

Here we will focus on the number of spinouts in different categories and the levels of external investment that they have received.

In [ ]:
spin.head()

In [ ]:
interesting_columns_spin = ['country_of_he_provider','region_of_he_provider','academic_year','metric','category_marker']

#check_categories(spin,interesting_columns_spin)

In [ ]:
#This creates a df with the number of active firms per category. This can be subset later

#Create a dict to filter the data
spin_n_filter = {'academic_year':'2017/18','metric':'Number of active firms'}

spins_number= multiple_nuts_estimates(filter_data(spin,spin_n_filter),uni_nuts,set(spin['category_marker']),'category_marker','value')

spins_number.columns = [x+'_n' for x in tidy_cols(spins_number)]

In [ ]:
spins_number.sort_values('graduate_start-ups_n',ascending=False).head()

In [ ]:
#This one focused on levels of investment received

#Create a dict to filter the data
spin_inv_filter = {'academic_year':'2017/18','metric':'Estimated external investment received (£ thousands)'}

spins_inv= multiple_nuts_estimates(filter_data(spin,spin_inv_filter),uni_nuts,set(spin['category_marker']),'category_marker','value')

spins_inv.columns = [x+'_inv_thGPB' for x in tidy_cols(spins_inv)]

In [ ]:
spins_inv.sort_values('staff_start-ups_inv_thGPB',ascending=False).head()

In [ ]:
spin_nuts = pd.concat([spins_number,spins_inv],axis=1)

### Licensing income related

We will extract total IP

In [ ]:
income.head()

In [ ]:
interesting_columns_income = ['country_of_he_provider','region_of_he_provider','academic_year','category_marker','unit']

#check_categories(income,interesting_columns_income)

In [ ]:
income_filter = {'academic_year':'2017/18','category_marker':'Total IP revenues'}

In [ ]:
income_nuts = make_nuts_estimate(filter_data(income,income_filter),uni_nuts,'value','total_ip_revenues')

In [ ]:
income_nuts.sort_values(ascending=False).head()

### services related

In [ ]:
services.head()

In [ ]:
interesting_columns = ['type_of_service','type_of_organisation','number/value_marker']

check_categories(services,interesting_columns)

In [ ]:
services_filter = {'academic_year':'2017/18','type_of_organisation':'Total','number/value_marker':'Value'}

services_nuts = multiple_nuts_estimates(filter_data(
    services,services_filter),uni_nuts,set(services['type_of_service']),'type_of_service','number/value')

services_nuts.columns = [x+'_thGBP' for x in tidy_cols(services_nuts)]

### Research collab related

In [ ]:
collab.head()

In [ ]:
interesting_columns_collab = ['academic_year','source_of_public_funding','type_of_income']

check_categories(collab,interesting_columns_collab)

None of these looks particularly relevant / new compared to information we have collected from other HESA sources - we will leave them out for now.

### Combine everything

In [ ]:
hebci_nuts = pd.concat([spin_nuts,income_nuts,services_nuts],axis=1)

In [ ]:
from scipy.stats import zscore

In [ ]:
sn.clustermap(hebci_nuts.apply(lambda x: zscore(x)).corr(),cmap='Oranges')

In [ ]:
hebci_nuts.to_csv(f'../../data/processed/hesa/{today_str}_hebci_nuts.csv')